In [7]:
using FFTW
using Printf
using LinearAlgebra
using Plots
using LaTeXStrings
γ=1.0
#mx=0.1
#my=10
N=300
dϕ = 2π/N
b = (1/(4π^2))*(dϕ/2)
Ef = 0.1
ħ=1
e=1
κ=1
nimp=1

function M(ϕ1,ϕ2,ratio,fermi)   
    mx = sqrt(ratio)
    my = 1/sqrt(ratio)
    W = 1 / ( ħ^2*cos(ϕ2)^2/(2*mx) + ħ^2*sin(ϕ2)^2/(2*my) )
    M = (2π/ħ) * (Overlap(ϕ1,ϕ2,ratio,fermi))^2 * W * nimp
  #  M = (2π/ħ) * W * nimp
    return M
end

function Overlap(ϕ1,ϕ2,ratio,fermi)
   mx = sqrt(ratio)
   my = 1/sqrt(ratio)
   q2 = (fermi/ħ^2)*( ( sqrt(2mx)*cos(ϕ1) - sqrt(2mx)*cos(ϕ2) )^2  + ( sqrt(2my)*sin(ϕ1) - sqrt(2my)*sin(ϕ2) )^2 )
   q = sqrt(q2)
   dos = sqrt(mx*my)/(π*ħ^2)
   kHk = 2π*e^2 / (q*κ + 2π*e^2*dos)
   return kHk
end

function tau_variational(ratio,fermi)
    result = 0.0
    mx = sqrt(ratio)
    my = 1/sqrt(ratio)
    b = 1/(4π^2)
    dos = sqrt(mx*my)/(π*ħ^2)
    for i=1:N, j=1:N
    ϕ1 = i*dϕ ; ϕ2 = j*dϕ
    integrand = (2π/ħ) * (cos(ϕ1) - cos(ϕ2))^2 * (Overlap(ϕ1,ϕ2,ratio,fermi))^2 * nimp
    result += integrand
  #  (2π/ħ) * (Overlap(ϕ1,ϕ2))^2 *
    end
    return 1/(result*b*dos*dϕ*dϕ /2) 
end

function tau_boltzmann(ratio,fermi)
    mx = sqrt(ratio)
    my = 1/sqrt(ratio)
#angles = collect(0 : dϕ : 2π)
    angles = [ϕ*2π/N for ϕ in 0:N-1]
    cosines = cos.(angles)
    Mij = [ M(ϕ1,ϕ2,ratio,fermi) for ϕ1 in angles, ϕ2 in angles ]
    ΣM = zeros(N)
    for i in 1:N
        ΣM[i] = sum(Mij[i,:])
    end       
    T1 = I(N) .* ΣM .* cosines        
    T2 = zeros(N,N)
    for i in 1:N, j in 1:N
        T2[i,j] = Mij[i,j] * cosines[j]
    end
   
    τ = zeros(N)
    T = (T1 - T2 + I*1e-10)
    τ = b^(-1)*inv(T)*cosines

    return τ
end


function rate_boltzmann(ratio,fermi)

τ = tau_boltzmann(ratio,fermi)
#τ_var = tau_variational(ratio,fermi)
#rate_var = 1/τ_var

result = τ[1]
 
    rate = zeros(N-2)
    angle = zeros(N-2)
    avg_rate = 0.0
    j=0
for i in 1:N
        ang = 2π*(i-1)/N * 180/π
        if (ang == 90) || (ang == 270)
            continue
        end
        j += 1
        angle[j] = 2π*(i-1)/N *1/π #* 180/π
        rate[j] = 1/τ[i]
        avg_rate += 1/τ[i] / (N-2)
#@printf("%5i %12.12f %8.4f %8.4f \n", i, 1/τ[i], ang, avg_rate )
end

return [angle,rate]
    
end


function plot_tau_angle(Ef)
rate_iso = 1 /tau_variational(1,Ef)
#(angle,rate) = rate_boltzmann(2,Ef)
#avg_rate = sum(rate)/size(rate)[1]

P = plot(legend=:right,minorgrid=true,xtickfontsize=18,ytickfontsize=18,xguidefontsize=18,yguidefontsize=18,legendfontsize=13)
    plot!(xlabel = L"Angle of incoming wave vector, $\phi_i$ (π)", ylabel = L"Scattering rate, $τ_i^{-1}/τ^{-1}_{\mathrm{iso}}$")
    plot!(rate_boltzmann(0.1,Ef)[1], rate_boltzmann(0.1,Ef)[2]/rate_iso, lw=2, label=L"$m_x/m_y = 0.1$"  )
    plot!(rate_boltzmann(0.2,Ef)[1], rate_boltzmann(0.2,Ef)[2]/rate_iso, lw=2,  label=L"m_x/m_y = 0.2"  )
    plot!(rate_boltzmann(0.5,Ef)[1], rate_boltzmann(0.5,Ef)[2]/rate_iso, lw=2, label=L"m_x/m_y = 0.5"  )
    plot!(rate_boltzmann(1,Ef)[1], rate_boltzmann(1,Ef)[2]/rate_iso, lw=2, label=L"m_x/m_y = 1"  )
    plot!(rate_boltzmann(2,Ef)[1], rate_boltzmann(2,Ef)[2]/rate_iso, lw=2, label=L"m_x/m_y = 2"  )
    plot!(rate_boltzmann(5,Ef)[1], rate_boltzmann(5,Ef)[2]/rate_iso, lw=2, label=L"m_x/m_y = 5"  )
    plot!(rate_boltzmann(10,Ef)[1], rate_boltzmann(10,Ef)[2]/rate_iso, lw=2, label=L"m_x/m_y = 10"  ) 
   return P 
end

function avg_rate(ratio,Ef)
(angle,rate) = rate_boltzmann(ratio,Ef)
avg_rate = sum(rate)/size(rate)[1]
    
return avg_rate
end

function main()    
    
Ef=0.1    
   P=plot_tau_angle(Ef)

    #    plot!(angle, [avg_rate/rate_iso for i=1:N-2], label="1/tau (avg)")
#    plot!(angle, [rate_var/rate_iso for i=1:N-2], label="1/tau (var)")
 #   dim = size([i for i=-2:0.1:2],1)
 #   avg_rate = zeros(dim)
#for i = -2 : 0.1 : 2
#    println(avg_rate(10^i,Ef))
#end


    
P2=plot(legend=:bottomleft,minorgrid=true,xtickfontsize=12,ytickfontsize=18,xguidefontsize=18,yguidefontsize=18,legendfontsize=18)
    plot!(xlabel = L"Mass anisotropy, $m_x / m_y$", ylabel = L"Scattering rate, $\langle τ^{-1} \rangle/τ^{-1}_{\mathrm{iso}}$")
Ef=0.1
rate_iso = 1 / tau_variational(1,Ef)
    plot!([10^i for i=-2:0.1:2], [(1/rate_iso)*(1/tau_variational(10^i,Ef)) for i=-2:0.1:2], xaxis=:log, ls=:dash, lw=3, label="variational" )
    plot!([10^i for i=-2:0.1:2], [(1/rate_iso)*avg_rate(10^i,Ef) for i=-2:0.1:2], xaxis=:log, lw=3, label="numerical")
plot(P)
#        savefig(P, "myplot1.pdf") 
        savefig(P, "plot1.png") 
        savefig(P2, "plot2.png") 

end

@time begin
   
main()
    
end

 12.097213 seconds (250.52 M allocations: 4.184 GiB, 4.29% gc time, 7.92% compilation time)
